## Imports


In [1]:
import warnings
from typing import Optional

import numpy as np
import pandas as pd
import pingouin as pg
import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio
import polars as pl
import polars.selectors as cs
from category_encoders import WOEEncoder
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.under_sampling import RandomUnderSampler
from lightgbm import LGBMClassifier
from optuna import Study, Trial, create_study
from optuna.pruners import SuccessiveHalvingPruner
from optuna.samplers import RandomSampler
from optuna.trial._frozen import FrozenTrial
from pandas import DataFrame, Series
from plotly.subplots import make_subplots
from polars import DataFrame
from skimpy import skim
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
#from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
#from cuml import RandomForestClassifier
from sklearn.impute import SimpleImputer

#from cuml.linear_model.logistic_regression import LogisticRegression
#from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model._logistic import LogisticRegression
from sklearn.metrics import (
    balanced_accuracy_score,
    confusion_matrix,
    fbeta_score,
    make_scorer,
    matthews_corrcoef,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    RepeatedStratifiedKFold,
    cross_val_score,
    train_test_split,
)
from polars.dataframe.frame import DataFrame
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from skopt import BayesSearchCV
from xgboost import XGBClassifier
from polars.series.series import Series
import scipy.stats as stats
from numpy.typing import NDArray
from optuna.study.study import Study


warnings.filterwarnings("ignore")
pio.templates.default = "plotly_dark"

#set trials globally to optimize run time
num_trials = 30

### Loan_functions.py

In [2]:
from loan_functions import (
    calculate_value_counts,
    clear,
    column_comparison,
    column_description,
    create_formatted_df,
    int_range,
    lower_column_names,
    lower_column_values,
    make_subplot,
    plot_histogram,
    null_count_comparison,
    objective,calculate_model_statistics, instantiate_model, 
    create_encoder_mapping, 
    encode_feature,
)

Let's begin by exploring the the training set. 

### Data Exploration


Let's start by reading in the training set and looking at balance. I'll keep the EDA of this notebook focused on the training set to have a more focused analysis and discussion of modeling and model performance, but a more detailed exploration of the supporting datasets can be found in the [notebook] found in this repository.

In [3]:
application_train = create_formatted_df("application_train.csv")

print(
    f"Training set has {application_train.shape[0]} rows and {application_train.shape[1]} columns."
)

application_train.sample(5)

Training set has 307511 rows and 122 columns.


sk_id_curr,target,name_contract_type,code_gender,flag_own_car,flag_own_realty,cnt_children,amt_income_total,amt_credit,amt_annuity,amt_goods_price,name_type_suite,name_income_type,name_education_type,name_family_status,name_housing_type,region_population_relative,days_birth,days_employed,days_registration,days_id_publish,own_car_age,flag_mobil,flag_emp_phone,flag_work_phone,flag_cont_mobile,flag_phone,flag_email,occupation_type,cnt_fam_members,region_rating_client,region_rating_client_w_city,weekday_appr_process_start,hour_appr_process_start,reg_region_not_live_region,reg_region_not_work_region,live_region_not_work_region,…,nonlivingarea_medi,fondkapremont_mode,housetype_mode,totalarea_mode,wallsmaterial_mode,emergencystate_mode,obs_30_cnt_social_circle,def_30_cnt_social_circle,obs_60_cnt_social_circle,def_60_cnt_social_circle,days_last_phone_change,flag_document_2,flag_document_3,flag_document_4,flag_document_5,flag_document_6,flag_document_7,flag_document_8,flag_document_9,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,flag_document_15,flag_document_16,flag_document_17,flag_document_18,flag_document_19,flag_document_20,flag_document_21,amt_req_credit_bureau_hour,amt_req_credit_bureau_day,amt_req_credit_bureau_week,amt_req_credit_bureau_mon,amt_req_credit_bureau_qrt,amt_req_credit_bureau_year
i64,i64,str,str,str,str,i64,f64,f64,f64,f64,str,str,str,str,str,f64,i64,i64,f64,i64,f64,i64,i64,i64,i64,i64,i64,str,f64,i64,i64,str,i64,i64,i64,i64,…,f64,str,str,f64,str,str,f64,f64,f64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64
436881,0,"""cash loans""","""m""","""y""","""y""",2,121500.0,180000.0,19516.5,180000.0,"""unaccompanied""","""working""","""secondary / secondary special""","""married""","""house / apartment""",0.028663,-10607,-2787,-1912.0,-3290,21.0,1,1,0,1,0,0,null,4.0,2,2,"""wednesday""",13,0,0,0,…,null,null,null,null,null,null,2.0,1.0,2.0,1.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,null,null,null,null,null,null
333510,0,"""cash loans""","""f""","""y""","""y""",1,202500.0,454500.0,14661.0,454500.0,"""unaccompanied""","""working""","""higher education""","""separated""","""house / apartment""",0.035792,-16749,-5303,-6173.0,-308,7.0,1,1,0,1,0,0,null,2.0,2,2,"""monday""",16,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-1805.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0
122944,0,"""cash loans""","""f""","""n""","""y""",0,157500.0,755190.0,36459.0,675000.0,"""unaccompanied""","""commercial associate""","""secondary / secondary special""","""civil marriage""","""house / apartment""",0.019689,-18783,-3336,-6915.0,-2303,null,1,1,0,1,0,0,"""laborers""",2.0,2,2,"""monday""",14,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
419870,0,"""cash loans""","""f""","""n""","""n""",1,135000.0,1.0782e6,31522.5,900000.0,"""unaccompanied""","""working""","""higher education""","""married""","""house / apartment""",0.022625,-16240,-3033,-2168.0,-4723,null,1,1,1,1,1,0,"""high skill tech staff""",3.0,2,2,"""wednesday""",10,0,0,0,…,0.0,"""reg oper account""","""block of flats""",0.079,"""stone, brick""","""no""",0.0,0.0,0.0,0.0,-638.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
392146,0,"""cash loans""","""f""","""n""","""n""",0,202500.0,656725.5,36801.0,594000.0,"""unaccompanied""","""working""","""secondary / secondary special""","""married""","""house / apartment""",0.008474,-19119,-7657,-11083.0,-2453,null,1,1,0,1,0,0,"""high skill tech staff""",2.0,2,2,"""thursday""",16,0,0,0,…,null,null,null,null,null,null,0.0,0.0,0.0,0.0,-2323.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,1.0,1.0,0.0,2.0


Let's now take a look at balance of the target variable since that is our modeling variable of interest. 

In [4]:
plot_histogram(application_train, "target", title="Target Variable Distribution")

In [5]:
description: DataFrame = pl.read_csv("HomeCredit_columns_description.csv", encoding="latin1")
description = lower_column_names(description)
description = lower_column_values(description)

# Delete training spaces for some entries in the row column
description = description.with_columns(
    pl.col("row").map_elements(lambda x: x.split(" ")[0], return_dtype=pl.String)
)

null_df = null_count_comparison(application_train, description, "amt_annuity")
null_df

table,feature,null_count
str,str,i64
"""application_{train|test}.csv""","""amt_annuity""",12
"""bureau.csv""","""amt_annuity""",42
"""previous_application.csv""","""amt_annuity""",93


### Null values 

In [34]:
null_df = (
    application_train.null_count()
    .transpose(include_header=True)
    .rename(mapping={"column": "feature", "column_0": "null_count"})
    .sort(by="null_count", descending=True)
    .with_columns(
        pl.col("null_count")
        .map_elements(lambda x: x / len(application_train), return_dtype=pl.Float32)
        .alias("percentage")
    )
)

px.histogram(
    null_df,
    x="percentage",
    text_auto=True,
    title="Null value percentages across dataset",
).update_layout(bargap=0.2).show()

The graph above shows that we have a pretty wide spread of null value prevalence within this dataset. Thankfully none of these null values are in the target variable, but we will have to make a choice in our analysis of how to treat null values within this dataset since some classifiers are more tolerant of null values than others. LightGBM and XGBoost can handle null values when fitting, but commonly used sklearn classifiers do not.

#### Look for categorical anomalies 

## Correlations 


In [7]:
corr: DataFrame = (
    application_train.select(cs.by_dtype(pl.NUMERIC_DTYPES)).to_pandas().corr()
)
corr = pd.DataFrame(corr["target"])

px.histogram(
    corr, "target", title="Correlations with Target Variable"
).update_layout(bargap=0.2)

The graph above shows the distribution of correlations as a graphical distribution rather than in tabular form because we have so many variables that we're working with. None of our variables are correlated with target, save for target itself.

## Hypothesis testing

### Is there difference in proportions of clients that default when broken down by their region of residence? 
$H_0$: $p_1$ = $p_2$ = $p_3$<br>
$H_1$: $p_1$ $\neq$ $p_2$ $\neq$ $p_3$


In [8]:
region_default: DataFrame = pd.crosstab(
    application_train["region_rating_client"].to_pandas(),
    application_train["target"].to_pandas(),
    rownames=["region_rating_client"],
    colnames=["target"],
)
region_default["default_proportion"] = region_default.iloc[:, 1] / region_default.sum(
    axis=1
)
region_default["total"] = region_default.iloc[:, :-1].sum(axis=1)

# pooled sample proportion
p1_default_prop:float = region_default.default_proportion.iloc[0]
p2_default_prop:float = region_default.default_proportion.iloc[1]
p3_default_prop:float = region_default.default_proportion.iloc[2]

p1_population:float = region_default.total.iloc[0]
p2_population:float = region_default.total.iloc[1]
p3_population:float = region_default.total.iloc[2]

p:float = (
    p1_default_prop * p1_population
    + p2_default_prop * p2_population
    + p3_default_prop * p3_population
) / (p1_population + p2_population + p3_population)

# standard error
se:float = np.sqrt(
    (p * (1 - p)) * ((1 / p1_population) + (1 / p2_population) + (1 / p2_population))
)

# test statistic
z:float = (p1_default_prop - p2_default_prop - p3_default_prop) / se

if np.abs(z) < 1.64485:
    print(
        f"Fail to reject the null hypothesis. We can assume the default percentage to be the same across {'region_rating_client'}."
    )
else:
    print(
        f"z = {z:.3f}. Reject null hypothesis. The proportion of credit defaults across values of {'region_rating_client'} is not equal."
    )

z = -82.387. Reject null hypothesis. The proportion of credit defaults across values of region_rating_client is not equal.


Because both gender and region are not equal across their values when compared to our target variable, we they are more likely to have a significant relationship with default risk. We should include these variables in our predictor pool. This raises the question of what variables we should use to predict our models, given that we feasibly cannot include all of them. Let's move on to feature selection so we can restrict our feature set to one that is both significant and predictive of the target variable without being so onerous to run calculations on. 


### Are younger homeowners are more likely to default on credit payments? 

$H_0$: $\mu_d$ =  $\mu_n$<br>
$H_1$:  $\mu_d$ $\neq$ $\mu_n$

s.t. <br>
$\mu_d$: the average age of clients who default<br>
$\mu_n$: the average age of clients who do not default

In [9]:
age_df: DataFrame = application_train[["target", "days_birth"]]
age_df = age_df.with_columns((pl.col("days_birth") // -365)).rename(
    {"days_birth": "age"}
)

age_default: Series = age_df.filter(pl.col("target") == 1)["age"]
age_no_default: Series = age_df.filter(pl.col("target") == 0)["age"]

t_stat, p_value = stats.ttest_ind(age_no_default, age_default)

print("Two-sample t-test results:\n")
print(f"t-statistic: {t_stat:.3f}")
print(f"p-value: {p_value:.3f}\n")

if p_value < 0.05:
    print(
        "Reject the null hypothesis: there is a significant difference in the average ages \nof clients who default vs. those who don't.\n"
    )
else:
    print(
        "Fail to reject the null hypothesis: there is no significant difference in the average ages of clients who \ndefault vs. those who don't.\n"
    )

Two-sample t-test results:

t-statistic: 43.517
p-value: 0.000

Reject the null hypothesis: there is a significant difference in the average ages 
of clients who default vs. those who don't.



## Preparing data for modeling


Before we start creating models, it's important to lay out what our north star assessment criteria are for model assessment, and secondarily how we will choose what a high-performing model is. 

Given that our business problem is designing predictive classifiers of credit clients that are likely to default on payments, and that our training data is highly imbalanced such that our positive class is the minority class at a ratio of 10:1, we will need to be very precise in our metric selection. 

Therefore each model that we create will be judged on its ROC-AUC score, F1 score, Matthews Correlation Coefficient, Balanced Accuracy, Precision, and Recall. This sounds like a long list of metrics, so our primary determinants will be the ROC-AUC and MCC since ROC-AUC can be less powerful for highly imbalanced datasets, and MCC gives a pretty comprehensive view of a model's performance across all 4 quadrants of the confusion matrix.

## Modeling

Now that we've defined our metrics set and evaluation method, our workflow for the next section is as follows: 
1. Create pipelines to impute missing values, scale data, and fit to our classifier of choice
2. Calculate model statistics for each classifier



In [10]:
application_train: DataFrame = create_formatted_df("application_train.csv")

x: DataFrame = application_train.drop(["sk_id_curr", "target"]).to_pandas()
y: DataFrame = pl.DataFrame(application_train["target"]).to_pandas()


x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.3, random_state=0, stratify=y
)

y_train: NDArray = np.array(y_train).ravel()

numerical_columns: list[str] = [
    *x.select_dtypes(exclude=["object", "category"]).columns
]

categorical_columns: list[str] = [
    *x.select_dtypes(include=["object", "category"]).columns
]

Now that we've defined the data that we'll be using for our modeling, we'll begin by taking a look at logistic regression.

In [11]:
model_performance = pd.DataFrame()
model_specifications = dict()


classifiers: list[str] = ["lgbm", "lgbm_rf", "xgboost"]

for classifier in classifiers:

    study = create_study(
        direction="maximize",
        study_name=(classifier + "\n"),
        pruner=SuccessiveHalvingPruner(reduction_factor=2),
        sampler=RandomSampler(seed=42),
    )

    study.optimize(
        lambda trial: objective(classifier, trial, x_train, np.array(y_train).ravel()),
        n_trials=num_trials,  # 4,
    )  # n_trials=100 is the original value

    model_specifications[classifier] = study.best_params

    best_trial: FrozenTrial = study.best_trial
    model: Pipeline = instantiate_model(
        classifier,
        trial=best_trial,
        numerical_columns=numerical_columns,
        categorical_columns=categorical_columns,
    )
    model.fit(x_train, y_train)
    predictions: NDArray = model.predict(x_test)
    model_performance[classifier] = calculate_model_statistics(y_test, predictions)
model_performance

[I 2024-12-31 13:05:14,492] A new study created in memory with name: lgbm

[I 2024-12-31 13:05:22,027] Trial 0 finished with value: 0.5015996625656801 and parameters: {'numerical_strategy': 'median', 'with_centering': True, 'with_scaling': False, 'categorical_strategy': 'constant', 'sigma': 0.10390188698471643, 'regularization': 4.8495492608099715, 'randomized': True, 'boosting_type': 'dart', 'num_leaves': 21, 'max_depth': 10, 'n_estimators': 85, 'class_weight': None, 'learning_rate': 0.13949386065204183}. Best is trial 0 with value: 0.5015996625656801.
[I 2024-12-31 13:05:27,500] Trial 1 finished with value: 0.6527110627714303 and parameters: {'numerical_strategy': 'constant', 'with_centering': False, 'with_scaling': True, 'categorical_strategy': 'most_frequent', 'sigma': 0.32619291333341227, 'regularization': 4.7444276862666666, 'randomized': True, 'boosting_type': 'gbdt', 'num_leaves': 45, 'max_depth': 8, 'n_estimators': 49, 'class_weight': 'balanced', 'learning_rate': 0.90932040207

,lgbm,lgbm_rf,xgboost
roc_auc,0.692193,0.676650,0.517876
matthews_correlation,0.225119,0.200341,0.044757
f_beta,0.518485,0.509180,0.085997
precision,0.170832,0.153276,0.133971
recall,0.669979,0.686224,0.082707
balanced_accuracy,0.692193,0.676650,0.517876


We're still not getting anywhere with these either, which means that our data could use a little more polishing before we feed them to models. One way to strengthen the models is with SMOTE oversampling, so we'll start there. 

## SMOTE Oversampling 
We'll use the combination of undersampling and oversampling originally put forth in the SMOTE paper to see how much that improves our model performance. We'll start by numerically encoding the categorical variables across application train before applying PCA and the conducting our under- and oversampling. 

In [12]:
numerical_train: DataFrame = application_train.clone()

# numerically encode features
encoder_mapping_key = dict()
for col in numerical_train.columns:
    try:
        key: dict[str, int] = create_encoder_mapping(numerical_train, col)
        numerical_train = encode_feature(numerical_train, col, key)
        encoder_mapping_key[col] = key
    except:
        pass


# fill missing values
for col in numerical_train.columns:
    median: float = numerical_train[col].median()
    numerical_train = numerical_train.with_columns(pl.col(col).fill_null(value=median))


x: DataFrame = numerical_train.drop("target").to_pandas()
y: Series = numerical_train["target"].to_pandas()

In [13]:
oversampling = SMOTE(sampling_strategy=0.1)
undersampling = RandomUnderSampler(sampling_strategy=0.5)

steps: list = [("oversample", oversampling), ("undersample", undersampling)]
pipeline = ImbPipeline(steps=steps)
smote_x, smote_y = pipeline.fit_resample(x, y)

pca = PCA(n_components=smote_x.shape[1])
smote_x: NDArray = pca.fit_transform(smote_x)

x_train, x_test, y_train, y_test = train_test_split(
    smote_x, smote_y, stratify=smote_y, random_state=15
)

Now that we've created our smote dataset, we can fit the models. However our previous model specifications aren't as helpful since the dataset is new, and models may perform better on different hyperparameters. Let's create another study and run it again! 

In [14]:
for idx, classifier in enumerate(['lgbm','lgbm_rf','xgboost']):
    study: Study = create_study(
        direction="maximize",
        study_name=(classifier+'\n'),
        pruner=SuccessiveHalvingPruner(reduction_factor=2),
        sampler=RandomSampler(seed=42),
    )
    study.optimize(lambda trial: objective(classifier, trial, x, y), n_trials=num_trials)


    model_specifications["smote_model"] = study.best_params

    if idx<=1:
        smote_model = LGBMClassifier(**study.best_params,device='gpu')
    else: 
        smote_model = XGBClassifier(**study.best_params, tree_method='gpu_hist',predictor='gpu_predictor', device='gpu'
                                    )
    smote_model.fit(x_train, y_train)
    predictions: NDArray = smote_model.predict(x_test)

    designator ="SMOTE_"+classifier

    model_performance[designator] = calculate_model_statistics(y_test, predictions)
model_performance

[I 2024-12-31 13:16:01,024] A new study created in memory with name: lgbm

[I 2024-12-31 13:16:08,675] Trial 0 finished with value: 0.5013967508317562 and parameters: {'numerical_strategy': 'median', 'with_centering': True, 'with_scaling': False, 'categorical_strategy': 'constant', 'sigma': 0.10390188698471643, 'regularization': 4.8495492608099715, 'randomized': True, 'boosting_type': 'dart', 'num_leaves': 21, 'max_depth': 10, 'n_estimators': 85, 'class_weight': None, 'learning_rate': 0.13949386065204183}. Best is trial 0 with value: 0.5013967508317562.
[I 2024-12-31 13:16:12,512] Trial 1 finished with value: 0.648381577180205 and parameters: {'numerical_strategy': 'constant', 'with_centering': False, 'with_scaling': True, 'categorical_strategy': 'most_frequent', 'sigma': 0.32619291333341227, 'regularization': 4.7444276862666666, 'randomized': True, 'boosting_type': 'gbdt', 'num_leaves': 45, 'max_depth': 8, 'n_estimators': 49, 'class_weight': 'balanced', 'learning_rate': 0.909320402078

,lgbm,lgbm_rf,xgboost,SMOTE_lgbm,SMOTE_lgbm_rf,SMOTE_xgboost
roc_auc,0.692193,0.676650,0.517876,0.690392,0.694142,0.625584
matthews_correlation,0.225119,0.200341,0.044757,0.362798,0.370168,0.305534
f_beta,0.518485,0.509180,0.085997,0.662951,0.666235,0.367973
precision,0.170832,0.153276,0.133971,0.531085,0.535940,0.636597
recall,0.669979,0.686224,0.082707,0.681760,0.684732,0.351493
balanced_accuracy,0.692193,0.676650,0.517876,0.690392,0.694142,0.625584
